In [ ]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
import time
from numpy import concatenate
import numpy as np
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from tensorflow.python.keras import Sequential
from math import sqrt;
import tensorflow as tf

In [ ]:
url = "https://drive.google.com/file/d/1u95FUEFI29NV-LmdCRHzRwj-t4tWH87m/view?usp=share_link"
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(['time', 'generation biomass', 
'generation fossil brown coal/lignite', 'generation fossil coal-derived gas', 'generation fossil hard coal','generation fossil oil shale',
'generation fossil peat',                            
'generation geothermal' ,                            
'generation hydro pumped storage aggregated',     
'generation hydro pumped storage consumption',       
'generation hydro run-of-river and poundage',                         
'generation marine',                                 
'generation nuclear',                                
'generation other',                                  
'generation other renewable',                        
'generation solar',                                  
'generation waste',                                  
'generation wind offshore',                         
'generation wind onshore',                                                     
'forecast wind onshore day ahead', 
'forecast wind offshore eday ahead',                  
'total load forecast',                                                               
'price day ahead',                                   
                                      ], axis=1)

In [ ]:
df.info()

In [ ]:
# for i in df.columns:
#   if df[i].isnull().sum() > 0:
#     df[i] = df[i].fillna(df[i].mean())

In [ ]:
df.isnull().sum()

In [ ]:
df.corr()['total load actual']

In [ ]:
df.columns

In [ ]:
values = df.values

In [ ]:
values.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
# scaledMin = scaler.fit_transform(valuesMin)
scaled = scaler.fit_transform(values)

In [ ]:
#covert to time-series
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
reframed = series_to_supervised(scaled, 1, 1)
reframed.head()

In [ ]:
reframed.shape

In [ ]:
reframed.drop(reframed.columns[[-1]], axis=1, inplace=True)

In [ ]:
reframed.shape

In [ ]:
reframed.drop(reframed.columns[[x for x in range(7, 10)]], axis=1, inplace=True)

In [ ]:
reframed.head()

In [ ]:
reframed.drop(reframed.columns[[-2]], axis=1, inplace=True)

In [ ]:
reframed.head()

In [ ]:
reframed.columns

In [ ]:
#split into train and test sets
values = reframed.values
# values5 = reframed5.values
n_train = round(values.shape[0])

train = values[:, :]
test = values[:, :]

In [ ]:
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [ ]:
#reshap input menjadi 3D ([)samples, timesteps, features)
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [ ]:
#design network LSTM
model = Sequential()
model.add(Bidirectional(LSTM(32, return_sequences = True, input_shape=(train_X.shape[1], train_X.shape[2]))))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(units = 64, return_sequences = True)))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(units = 64, return_sequences = True)))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(units = 64)))
model.add(Dropout(0.2))

model.add(Dense(1, activation='tanh'))

model.compile(loss='mse', optimizer='adam')

In [ ]:
#fit network
start = time.time()
history = model.fit(train_X, train_y, epochs=500, batch_size=128, validation_data=(test_X, test_y), verbose=1, shuffle=False)
end = time.time()
print('Processing Time {} seconds.'.format(end-start))

In [ ]:
#plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.xlabel('Epoch')
pyplot.ylabel('Loss')
pyplot.legend()
pyplot.show()

In [ ]:
#membuat prediksi training
xhat = model.predict(train_X)
train_X = train_X.reshape((train_X.shape[0], train_X.shape[2]))

#invert scaling untuk peramalan
inv_xhat = concatenate((xhat, train_X[:, 1:]), axis=1)
inv_xhat = scaler.inverse_transform(inv_xhat)
inv_xhat = inv_xhat[:,0]

#invert scaling untuk aktual
train_y = train_y.reshape((len(train_y), 1))
inv_x = concatenate((train_y, train_X[:, 1:]), axis=1)
inv_x = scaler.inverse_transform(inv_x)
inv_x = inv_x[:,0]

#================================================================#

#membuat prediksi testing
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

#invert scaling untuk peramalan
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

#invert scaling untuk aktual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [ ]:
# calculate RMSE
#rmsetrain = np.sqrt(mean_squared_error(xhat, train_y))
#print(f'RMSE Training is : %.3f' % rmsetrain)
#rmsetrain = np.sqrt(mean_squared_error(inv_x, inv_xhat))
#print(f'RMSE Training is : %.3f' % rmsetrain)

rmsetest = np.sqrt(mean_squared_error(test_y, yhat))
print(f'RMSE Testing is : %.3f' % rmsetest)
#rmsetest = np.sqrt(mean_squared_error(inv_y, inv_yhat))
#print(f'RMSE Testing is : %.3f' % rmsetest)


def mean_absolute_percentage_error(yhat, test_y):
    yhat, test_y = np.array(yhat), np.array(test_y)
    return np.mean(np.abs((yhat, test_y) / test_y)) 

#print(f'MAPE train is : {(mean_absolute_percentage_error(train_y, xhat))}')
#print(f'MAPE train is : {(mean_absolute_percentage_error(inv_x, inv_xhat))}')

print(f'MAPE test is : {(mean_absolute_percentage_error(test_y, yhat))}')
#print(f'MAPE test is : {(mean_absolute_percentage_error(inv_y, inv_yhat))}')

#print(f'R2 train is : {r2_score(train_y, xhat)}')
#print(f'R2 train is : {r2_score(inv_x, inv_xhat)}')

print(f'R2 test is : {r2_score(test_y, yhat)}')
#print(f'R2 test is : {r2_score(inv_y, inv_yhat)}')


In [ ]:
print('Actual :', test_y)
print('Predicted:', yhat)
# plot history
pyplot.plot(test_y, label='Actual')
pyplot.plot(yhat, label='Forecasting')
pyplot.xlabel('Timestep')
pyplot.ylabel('Value')
pyplot.legend()
pyplot.show()

In [ ]:
idx = 200#int(len(yhat)*0.25)
aa=[x for x in range(idx)]
pyplot.figure(figsize=(20,4))
pyplot.plot(aa, test_y[:idx], marker='.', label="actual")
pyplot.plot(aa, yhat[:idx], 'r', label="prediction")
# plt.tick_params(left=False, labelleft=True) #remove ticks
pyplot.tight_layout()
sns.despine(top=True)
pyplot.subplots_adjust(left=0.07)
pyplot.ylabel('TOTAL Load', size=15)
pyplot.xlabel('Time step', size=15)
pyplot.legend(fontsize=15)
pyplot.show()